In [14]:
import altair as alt
import pandas as pd
import numpy as np
import sys
sys.path.append('../src')

from data_collection import FinancialDataCollector
from data_processing import FinancialDataProcessor
from config import SECTOR_MAPPING

alt.renderers.enable('jupyterlab')  # For JupyterLab

RendererRegistry.enable('jupyterlab')

In [11]:
# Load data
collector = FinancialDataCollector()
stock_data = collector.load_stock_data()
processor = FinancialDataProcessor(stock_data)

Data loaded from data\raw\stock_data.pkl


In [ ]:
%pip install altair_viewer

In [ ]:
# Plot-1. Multi-faceted Stock Analysis
def create_altair_multifacet():
    # Prepare data
    stock_list = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'NVDA', 'META']
    data_for_altair = []
    
    for ticker in stock_list:
        prices = stock_data[ticker]['prices']
        for date, row in prices.iterrows():
            data_for_altair.append({
                'Date': date.strftime('%Y-%m-%d'),
                'Stock': ticker,
                'Close': row['Close'],
                'Volume': row['Volume'],
                'Returns': row['Returns'] * 100 if not pd.isna(row['Returns']) else 0
            })
    
    df = pd.DataFrame(data_for_altair)
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Base chart
    base = alt.Chart(df).add_selection(
        alt.selection_interval(bind='scales', encodings=['x'])
    )
    
    # Price chart
    price_chart = base.mark_line(strokeWidth=2).encode(
        x=alt.X('Date:T', title='Date'),
        y=alt.Y('Close:Q', title='Stock Price ($)', scale=alt.Scale(type='log')),
        color=alt.Color('Stock:N', scale=alt.Scale(scheme='category10')),
        tooltip=['Date:T', 'Stock:N', 'Close:Q']
    ).resolve_scale(y='independent').properties(
        width=600,
        height=200,
        title='Stock Prices (Log Scale)'
    )
    
    # Volume chart
    volume_chart = base.mark_bar(opacity=0.7).encode(
        x=alt.X('Date:T', title='Date'),
        y=alt.Y('Volume:Q', title='Volume'),
        color=alt.Color('Stock:N', scale=alt.Scale(scheme='category10')),
        tooltip=['Date:T', 'Stock:N', 'Volume:Q']
    ).resolve_scale(y='independent').properties(
        width=600,
        height=150,
        title='Trading Volume'
    )
    
    # Returns distribution
    returns_hist = alt.Chart(df).mark_bar(opacity=0.7).encode(
        x=alt.X('Returns:Q', bin=alt.Bin(maxbins=50), title='Daily Returns (%)'),
        y=alt.Y('count()', title='Frequency'),
        color=alt.Color('Stock:N', scale=alt.Scale(scheme='category10')),
        column=alt.Column('Stock:N', title='Stock'),
        tooltip=['Stock:N', 'count()']
    ).resolve_scale(y='independent').properties(
        width=120,
        height=100,
        title='Returns Distribution by Stock'
    )
    
    # Combine charts
    combined = alt.vconcat(
        price_chart,
        volume_chart,
        returns_hist
    ).resolve_scale(color='independent')
    
    combined.save('./static/altair_multifacet.html')
    
    return combined

multifacet_chart = create_altair_multifacet()

In [ ]:
# Plot-2. Interactive Correlation Matrix

def create_altair_correlation():
    # Get correlation data
    selected_stocks = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'NVDA', 'META', 'AMZN', 'JPM']
    correlation_matrix = processor.calculate_correlation_matrix(selected_stocks)
    
    # Reshape correlation matrix for Altair
    corr_data = []
    for i, stock1 in enumerate(correlation_matrix.index):
        for j, stock2 in enumerate(correlation_matrix.columns):
            corr_data.append({
                'Stock1': stock1,
                'Stock2': stock2,
                'Correlation': correlation_matrix.iloc[i, j],
                'x': j,
                'y': i
            })
    
    corr_df = pd.DataFrame(corr_data)
    
    # Create heatmap
    heatmap = alt.Chart(corr_df).mark_rect().encode(
        x=alt.X('Stock2:N', title='Stock'),
        y=alt.Y('Stock1:N', title='Stock'),
        color=alt.Color('Correlation:Q', 
                       scale=alt.Scale(scheme='redblue', domain=[-1, 1]),
                       title='Correlation'),
        tooltip=['Stock1:N', 'Stock2:N', 'Correlation:Q']
    ).properties(
        width=400,
        height=400,
        title='Stock Return Correlations'
    )
    
    # Add text annotations
    text = alt.Chart(corr_df).mark_text(
        baseline='middle',
        fontSize=10,
        fontWeight='bold'
    ).encode(
        x='Stock2:N',
        y='Stock1:N',
        text=alt.Text('Correlation:Q', format='.2f'),
        color=alt.condition(alt.datum.Correlation > 0, alt.value('white'), alt.value('black'))
    )
    
    correlation_chart = (heatmap + text)
    correlation_chart.save('./static/altair_correlation.html')
    
    return correlation_chart

correlation_chart = create_altair_correlation()

In [ ]:
# Plot-3. Risk-Return Scatter with Sector Encoding
def create_altair_risk_return():
    # Calculate risk-return metrics
    risk_return_data = []
    
    for ticker in list(stock_data.keys())[:25]:
        returns = stock_data[ticker]['prices']['Returns'].dropna()
        annual_return = returns.mean() * 252 * 100
        annual_volatility = returns.std() * np.sqrt(252) * 100
        market_cap = stock_data[ticker]['info'].get('marketCap', 0) / 1e9
        
        # Get sector
        sector = 'Other'
        for sect, tickers in SECTOR_MAPPING.items():
            if ticker in tickers:
                sector = sect
                break
        
        risk_return_data.append({
            'Ticker': ticker,
            'Annual_Return': annual_return,
            'Annual_Volatility': annual_volatility,
            'Market_Cap': market_cap,
            'Sector': sector
        })
    
    rr_df = pd.DataFrame(risk_return_data)
    
    # Selection for highlighting
    highlight = alt.selection(type='single', on='mouseover')
    
    # Base chart
    base = alt.Chart(rr_df).add_selection(highlight)
    
    # Scatter plot
    points = base.mark_circle(size=100).encode(
        x=alt.X('Annual_Volatility:Q', title='Annual Volatility (%)', scale=alt.Scale(zero=False)),
        y=alt.Y('Annual_Return:Q', title='Annual Return (%)', scale=alt.Scale(zero=False)),
        color=alt.Color('Sector:N', scale=alt.Scale(scheme='category10')),
        size=alt.Size('Market_Cap:Q', scale=alt.Scale(range=[50, 400]), title='Market Cap (B$)'),
        opacity=alt.condition(highlight, alt.value(0.8), alt.value(0.6)),
        stroke=alt.condition(highlight, alt.value('black'), alt.value(None)),
        strokeWidth=alt.condition(highlight, alt.value(2), alt.value(0)),
        tooltip=['Ticker:N', 'Annual_Return:Q', 'Annual_Volatility:Q', 'Market_Cap:Q', 'Sector:N']
    ).properties(
        width=500,
        height=400,
        title='Risk vs Return by Sector'
    )
    
    # Add labels for selected points
    text = base.mark_text(
        align='left',
        baseline='middle',
        dx=7,
        fontSize=10,
        fontWeight='bold'
    ).encode(
        x='Annual_Volatility:Q',
        y='Annual_Return:Q',
        text=alt.condition(highlight, 'Ticker:N', alt.value('')),
        color=alt.value('black')
    )
    
    # Reference lines
    rule_x = alt.Chart(rr_df).mark_rule(color='gray', strokeDash=[3, 3]).encode(
        x=alt.X('mean(Annual_Volatility):Q')
    )
    
    rule_y = alt.Chart(rr_df).mark_rule(color='gray', strokeDash=[3, 3]).encode(
        y=alt.Y('mean(Annual_Return):Q')
    )
    
    risk_return_chart = (points + text + rule_x + rule_y)
    risk_return_chart.save('./static/altair_risk_return.html')
        
    return risk_return_chart

risk_return_chart = create_altair_risk_return()

In [ ]:
# Plot-4. Time Series with Brushing and Linking
def create_altair_brushing():
    # Prepare data for FAANG stocks
    faang_stocks = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL']
    ts_data = []
    
    for ticker in faang_stocks:
        prices = stock_data[ticker]['prices']
        normalized_prices = (prices['Close'] / prices['Close'].iloc[0]) * 100
        
        for date, price in normalized_prices.items():
            ts_data.append({
                'Date': date.strftime('%Y-%m-%d'),
                'Stock': ticker,
                'Normalized_Price': price,
                'Volume': prices.loc[date, 'Volume']
            })
    
    ts_df = pd.DataFrame(ts_data)
    ts_df['Date'] = pd.to_datetime(ts_df['Date'])
    
    # Brush selection
    brush = alt.selection(type='interval', encodings=['x'])
    
    # Upper chart with brush
    upper = alt.Chart(ts_df).mark_line(strokeWidth=2).encode(
        x=alt.X('Date:T', title='Date'),
        y=alt.Y('Normalized_Price:Q', title='Normalized Price (Base=100)', scale=alt.Scale(zero=False)),
        color=alt.Color('Stock:N', scale=alt.Scale(scheme='category10')),
        opacity=alt.condition(brush, alt.value(0.8), alt.value(0.4)),
        tooltip=['Date:T', 'Stock:N', 'Normalized_Price:Q']
    ).add_selection(
        brush
    ).properties(
        width=600,
        height=300,
        title='FAANG Stocks Performance (Brush to Filter)'
    )
    
    # Lower chart filtered by brush
    lower = alt.Chart(ts_df).mark_circle(size=30).encode(
        x=alt.X('Date:T', title='Date'),
        y=alt.Y('Volume:Q', title='Volume', scale=alt.Scale(type='log')),
        color=alt.Color('Stock:N', scale=alt.Scale(scheme='category10')),
        tooltip=['Date:T', 'Stock:N', 'Volume:Q']
    ).transform_filter(
        brush
    ).properties(
        width=600,
        height=200,
        title='Volume (Filtered by Selection Above)'
    )
    
    brushing_chart = alt.vconcat(upper, lower)
    brushing_chart.save('./static/altair_brushing.html')
    
    return brushing_chart

brushing_chart = create_altair_brushing()

In [ ]:
# Plot-5. Sector Analysis Dashboard
def create_altair_sector_dashboard():
    # Prepare sector data
    sector_data = []
    
    for sector, tickers in SECTOR_MAPPING.items():
        sector_returns = []
        sector_volatilities = []
        
        for ticker in tickers:
            if ticker in stock_data:
                returns = stock_data[ticker]['prices']['Returns'].dropna()
                annual_return = returns.mean() * 252 * 100
                annual_volatility = returns.std() * np.sqrt(252) * 100
                market_cap = stock_data[ticker]['info'].get('marketCap', 0) / 1e9
                
                sector_data.append({
                    'Sector': sector,
                    'Stock': ticker,
                    'Annual_Return': annual_return,
                    'Annual_Volatility': annual_volatility,
                    'Market_Cap': market_cap
                })
    
    sector_df = pd.DataFrame(sector_data)
    
    # Sector performance bar chart
    sector_avg = sector_df.groupby('Sector').agg({
        'Annual_Return': 'mean',
        'Annual_Volatility': 'mean',
        'Market_Cap': 'sum'
    }).reset_index()
    
    bars = alt.Chart(sector_avg).mark_bar().encode(
        x=alt.X('Sector:N', sort='-y', title='Sector'),
        y=alt.Y('Annual_Return:Q', title='Average Annual Return (%)'),
        color=alt.Color('Annual_Return:Q', scale=alt.Scale(scheme='viridis'), title='Return (%)'),
        tooltip=['Sector:N', 'Annual_Return:Q', 'Annual_Volatility:Q', 'Market_Cap:Q']
    ).properties(
        width=400,
        height=200,
        title='Average Sector Performance'
    )
    
    # Individual stock scatter
    scatter = alt.Chart(sector_df).mark_circle(size=60).encode(
        x=alt.X('Annual_Volatility:Q', title='Annual Volatility (%)'),
        y=alt.Y('Annual_Return:Q', title='Annual Return (%)'),
        color=alt.Color('Sector:N', scale=alt.Scale(scheme='category10')),
        size=alt.Size('Market_Cap:Q', scale=alt.Scale(range=[20, 200])),
        tooltip=['Stock:N', 'Sector:N', 'Annual_Return:Q', 'Annual_Volatility:Q', 'Market_Cap:Q']
    ).properties(
        width=400,
        height=300,
        title='Individual Stock Risk-Return'
    )
    
    # Market cap distribution
    market_cap_chart = alt.Chart(sector_avg).mark_arc(innerRadius=20).encode(
        theta=alt.Theta('Market_Cap:Q', title='Total Market Cap'),
        color=alt.Color('Sector:N', scale=alt.Scale(scheme='category10')),
        tooltip=['Sector:N', 'Market_Cap:Q']
    ).properties(
        width=200,
        height=200,
        title='Market Cap by Sector'
    )
    
    # Combine charts
    top_row = alt.hconcat(bars, market_cap_chart)
    sector_dashboard = alt.vconcat(top_row, scatter)
    
    sector_dashboard.save('./static/altair_sector_dashboard.html')
    
    return sector_dashboard

sector_dashboard = create_altair_sector_dashboard()
